In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy
import threading
import python_speech_features
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import IPython.display as ipd
import math
%matplotlib inline
import librosa
import pickle
from os import listdir
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Dropout, LSTM, Conv2D, MaxPooling2D, Flatten, ConvLSTM1D
from os.path import isfile, join

inputShape = (25, 20)

model = Sequential()

model.add(SimpleRNN(units = 50, activation='tanh', return_sequences=True, input_shape= inputShape))
model.add(Dropout(0.2))

model.add(SimpleRNN(units = 30, activation='tanh', return_sequences=True))
model.add(Dropout(0.2))

model.add(SimpleRNN(units = 20, activation='tanh', return_sequences=True))
model.add(Dropout(0.2))

model.add(SimpleRNN(units = 10))
model.add(Dropout(0.2))

model.add(Dense(units = 1))

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_4 (SimpleRNN)     (None, 25, 50)            3550      
_________________________________________________________________
dropout_4 (Dropout)          (None, 25, 50)            0         
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, 25, 30)            2430      
_________________________________________________________________
dropout_5 (Dropout)          (None, 25, 30)            0         
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 25, 20)            1020      
_________________________________________________________________
dropout_6 (Dropout)          (None, 25, 20)            0         
_________________________________________________________________
simple_rnn_7 (SimpleRNN)     (None, 10)               

In [2]:
import pickle
def readObjectsFromFile(fName, objects):
    with (open(fName, "rb")) as openfile:
        while True:
            try:
                objects.append(pickle.load(openfile))
            except EOFError:
                break

objects = []
readObjectsFromFile("MNIST_db8_3_train_test.pickle", objects)
# model.compile(optimizer='adam', loss='mean_squared_error')
# model.fit(objects[0]["x_train"], y_train, epochs=10, batch_size=32)

2021-11-25 13:51:52.779223: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-25 13:51:53.531376: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 517 MB memory:  -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:da:00.0, compute capability: 7.5


In [14]:
objects[0]["x_train"] = np.asarray(objects[0]["x_train"])

In [18]:
objects[0]["y_train"]

array([[2],
       [2],
       [2],
       ...,
       [8],
       [8],
       [8]])

In [36]:
model.fit(xTrain, yTrain, epochs=10, batch_size=32)

Epoch 1/10
1173/1173 [==============================] - 97s 81ms/step - loss: 9.8763
Epoch 2/10
1173/1173 [==============================] - 97s 82ms/step - loss: 8.7719
Epoch 3/10
1173/1173 [==============================] - 96s 82ms/step - loss: 8.6012
Epoch 4/10
1173/1173 [==============================] - 97s 83ms/step - loss: 8.5195
Epoch 5/10
1173/1173 [==============================] - 97s 83ms/step - loss: 8.3208
Epoch 6/10
1173/1173 [==============================] - 96s 82ms/step - loss: 8.4395
Epoch 7/10
1173/1173 [==============================] - 96s 82ms/step - loss: 8.6040
Epoch 8/10
1173/1173 [==============================] - 96s 82ms/step - loss: 8.2247
Epoch 9/10
1173/1173 [==============================] - 99s 84ms/step - loss: 8.0415
Epoch 10/10
1173/1173 [==============================] - 98s 84ms/step - loss: 7.9977


In [41]:
objects[0]["x_test"] = np.asarray(objects[0]["x_test"])

In [42]:
xTest = np.empty((953*4, 25, 20))
xInd = 0
yTest = np.empty((953*4,1))
for i in range(len(objects[0]["x_test"])):
    xTest[xInd] = objects[0]["x_test"][i,:,0,:]
    xTest[xInd+1] = objects[0]["x_test"][i,:,1,:]
    xTest[xInd+2] = objects[0]["x_test"][i,:,2,:]
    xTest[xInd+3] = objects[0]["x_test"][i,:,3,:]
    yTest[xInd] = objects[0]["y_test"][i]
    yTest[xInd+1] = objects[0]["y_test"][i]
    yTest[xInd+2] = objects[0]["y_test"][i]
    yTest[xInd+3] = objects[0]["y_test"][i]
    xInd+=4

In [45]:
preds = model.predict(xTest)

In [46]:
len(preds)

3812

In [51]:
tot = 0
for i in range(3812):
    if np.round(yTest[i]-preds[i]) != 0:
        tot+=1

In [52]:
tot/3812

0.8753934942287513